In [ ]:
from rnamodif.architectures.rodan_event import RodanEvent
from rnamodif.data_utils.dataloading_events import event_datamodule

from rnamodif.data_utils.split_methods import get_kfold_splits, get_fullvalid_split, get_valid_portions
import pytorch_lightning as pl
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning.callbacks import ModelCheckpoint

#TODO low GPU utilization - fix bottlenecks
dm = event_datamodule(
    readers_split_config={
        'pos':[
            {'reader_path':'/home/jovyan/taiyaki_test/mapped_reads_2/m6A_33', 'train_portion':0.8},
            # {'reader_path':'/home/jovyan/taiyaki_test/mapped_reads_2/RNAAB090763_m6A', 'train_portion':0.0},
            # {'reader_path':'/home/jovyan/taiyaki_test/mapped_reads_2/m6A_10', 'train_portion':0.0},
            # {'reader_path':'/home/jovyan/taiyaki_test/mapped_reads_2/m6A_5', 'train_portion':0.0},
        ],
        'neg':[
            {'reader_path':'/home/jovyan/taiyaki_test/mapped_reads_2/m6A_0', 'train_portion':0.8},
            # {'reader_path':'/home/jovyan/taiyaki_test/mapped_reads_2/RNAAB089716_m6A_UNM', 'train_portion':0.0},
        ],
    },
    batch_size=64, 
    workers=2,
    window=4096,
    valid_limit = 5000,
)


model = RodanEvent(lr=1e-4, warmup_steps=2500, loss_type='mil', lossweight=1)

experiment_name = 'm6a_0v33_MIL_sampleavg_smooth5_blank03'
checkpoint_callback = ModelCheckpoint(
    dirpath=f"/home/jovyan/RNAModif/rnamodif/checkpoints_pl/{experiment_name}", 
    save_top_k=2, 
    monitor="valid_loss", 
    save_last=True, 
    save_weights_only=False
)

logger = CometLogger(api_key="TEVQbgxxvilM1WdTyqZLJ57ac", project_name='RNAModif', experiment_name=experiment_name) 
trainer= pl.Trainer(
    max_steps = 1000000, logger=logger, accelerator='gpu',
    auto_lr_find=False, val_check_interval=500,  
    log_every_n_steps=500, benchmark=True, precision=16,
    callbacks=[checkpoint_callback], #commented out = no saving
    # profiler='simple')
    # resume_from_checkpoint=f'/home/jovyan/RNAModif/rnamodif/checkpoints_pl/{experiment_name}/last.ckpt'
)

trainer.fit(model, dm)

CometLogger will be initialized in online mode


using MIL loss


Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


vocab ok


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type           | Params
----------------------------------------------
0 | rodan      | network        | 10.7 M
1 | mod_neuron | Linear         | 769   
2 | acc        | BinaryAccuracy | 0     
----------------------------------------------
10.7 M    Trainable params
0         Non-trainable params
10.7 M    Total params
21.341    Total estimated model params size (MB)


...

/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  return bound(*args, **kwds)
/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/numpy/core/fromnumeric.py:57: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return bound(*args, **kwds)
/home/jovyan/my-conda-en

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]